<a href="https://colab.research.google.com/github/giusscr/test-long-short-portfolio/blob/main/test_to_send_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPortfolioOpt
!pip install yfinance
!pip install cvxpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 724.8 kB/s eta 0:00:00


In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
from pypfopt import objective_functions
import numpy as np
import pandas as pd

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


**Download Stock Price Data**

In [ ]:
stocks = ["ADBE", "AMZN", "ENPH", "FTNT", "MCD", "PFE"]
stock_data = yf.download( stocks, start="2020-10-1", end="2024-01-5")["Adj Close"]

[*********************100%%**********************]  6 of 6 completed


In [ ]:
stock_data

Ticker,ADBE,AMZN,ENPH,FTNT,MCD,PFE
Date,,,,,,
2020-10-01,499.510010,161.063004,89.510002,24.049999,204.110550,30.002163
2020-10-02,478.989990,156.250000,90.430000,23.275999,206.973450,30.010414
2020-10-05,486.470001,159.960007,98.050003,23.818001,210.133743,30.315632
2020-10-06,478.980011,154.998001,98.570000,23.660000,208.293320,29.837177
2020-10-07,493.149994,159.784500,107.820000,24.138000,210.514862,30.084652
...,...,...,...,...,...,...
2023-12-28,595.520020,153.380005,134.789993,59.400002,295.839996,28.363180
2023-12-29,596.599976,151.940002,132.139999,58.529999,296.510010,28.363180
2024-01-02,580.070007,149.929993,131.240005,57.779999,297.040009,29.289244


##A)STOCK RETURNS

###1)ORIGINAL STOCK RETURNS

In [ ]:
stock_returns = stock_data.pct_change()[1:]
stock_returns

Ticker,ADBE,AMZN,ENPH,FTNT,MCD,PFE
Date,,,,,,
2020-10-02,-0.041080,-0.029883,0.010278,-0.032183,0.014026,0.000275
2020-10-05,0.015616,0.023744,0.084264,0.023286,0.015269,0.010170
2020-10-06,-0.015397,-0.031020,0.005303,-0.006634,-0.008758,-0.015782
2020-10-07,0.029584,0.030881,0.093842,0.020203,0.010665,0.008294
2020-10-08,-0.004684,-0.001608,-0.021517,0.007871,-0.003002,0.011516
...,...,...,...,...,...,...
2023-12-28,-0.000939,0.000261,-0.006193,0.001011,0.004380,0.006291
2023-12-29,0.001813,-0.009388,-0.019660,-0.014647,0.002265,0.000000
2024-01-02,-0.027707,-0.013229,-0.006811,-0.012814,0.001787,0.032650


###2)MODIFIED STOCK RETURNS

In [ ]:
stock_returns_inverted = stock_returns.copy()

# Cambia il segno solo nelle colonne ADBE e PFE nel nuovo DataFrame
stock_returns_inverted[['ADBE', 'PFE']] = -stock_returns_inverted[['ADBE', 'PFE']]

# Visualizza il nuovo DataFrame
stock_returns_inverted

Ticker,ADBE,AMZN,ENPH,FTNT,MCD,PFE
Date,,,,,,
2020-10-02,0.041080,-0.029883,0.010278,-0.032183,0.014026,-0.000275
2020-10-05,-0.015616,0.023744,0.084264,0.023286,0.015269,-0.010170
2020-10-06,0.015397,-0.031020,0.005303,-0.006634,-0.008758,0.015782
2020-10-07,-0.029584,0.030881,0.093842,0.020203,0.010665,-0.008294
2020-10-08,0.004684,-0.001608,-0.021517,0.007871,-0.003002,-0.011516
...,...,...,...,...,...,...
2023-12-28,0.000939,0.000261,-0.006193,0.001011,0.004380,-0.006291
2023-12-29,-0.001813,-0.009388,-0.019660,-0.014647,0.002265,-0.000000
2024-01-02,0.027707,-0.013229,-0.006811,-0.012814,0.001787,-0.032650


##B)EXPECTED RETURNS

##1) Expected Returns ORIGINAL

In [ ]:

#"ADBE", "AMZN", "ENPH", "FTNT", "MCD", "PFE"
er= [-0.15, 0.08, 0.30, 0.14, 0.12, -0.20]

# Creazione del DataFrame con i rendimenti attesi
expected_returns_df = pd.DataFrame(index=stocks, data={"expected_returns": er})
expected_returns_series = expected_returns_df["expected_returns"]

# Visualizza la Serie di rendimenti attesi
print(expected_returns_series)


ADBE   -0.15
AMZN    0.08
ENPH    0.30
FTNT    0.14
MCD     0.12
PFE    -0.20
Name: expected_returns, dtype: float64


##2)EXPECTED RETURNS ALL POSITIVE

In [ ]:
er_inverted = [0.15, 0.08, 0.30, 0.14, 0.12, 0.20]


# Creazione del DataFrame con i rendimenti attesi
expected_returns_df = pd.DataFrame(index=stocks, data={"expected_returns": er_inverted})
expected_returns_series_inverted = expected_returns_df["expected_returns"]

# Visualizza la Serie di rendimenti attesi
print(expected_returns_series_inverted)

ADBE    0.15
AMZN    0.08
ENPH    0.30
FTNT    0.14
MCD     0.12
PFE     0.20
Name: expected_returns, dtype: float64


##C) Variance-Covariance Matrix

##ORIGINAL MATRIX

In [ ]:
cov_matrix = risk_models.sample_cov(stock_data)
print(cov_matrix)

Ticker      ADBE      AMZN      ENPH      FTNT       MCD       PFE
Ticker                                                            
ADBE    0.127974  0.081290  0.091984  0.088622  0.019089  0.011134
AMZN    0.081290  0.137301  0.081160  0.072140  0.017446  0.010137
ENPH    0.091984  0.081160  0.454314  0.105865  0.018498  0.004934
FTNT    0.088622  0.072140  0.105865  0.190570  0.020031  0.018947
MCD     0.019089  0.017446  0.018498  0.020031  0.027472  0.006382
PFE     0.011134  0.010137  0.004934  0.018947  0.006382  0.068420


##MODIFIED MATRIX (I CALL IT COV_MATRIX_INVERTED)

In [ ]:
cov_matrix_inverted = risk_models.sample_cov(stock_returns_inverted,returns_data=True)
print(cov_matrix_inverted)


Ticker      ADBE      AMZN      ENPH      FTNT       MCD       PFE
Ticker                                                            
ADBE    0.127974 -0.081290 -0.091984 -0.088622 -0.019089  0.011134
AMZN   -0.081290  0.137301  0.081160  0.072140  0.017446 -0.010137
ENPH   -0.091984  0.081160  0.454314  0.105865  0.018498 -0.004934
FTNT   -0.088622  0.072140  0.105865  0.190570  0.020031 -0.018947
MCD    -0.019089  0.017446  0.018498  0.020031  0.027472 -0.006382
PFE     0.011134 -0.010137 -0.004934 -0.018947 -0.006382  0.068420


##ORIGINAL CORRELATION

In [ ]:
corr_matrix = risk_models.cov_to_corr(cov_matrix)
print(corr_matrix)

Ticker      ADBE      AMZN      ENPH      FTNT       MCD       PFE
Ticker                                                            
ADBE    1.000000  0.613254  0.381484  0.567486  0.321937  0.118987
AMZN    0.613254  1.000000  0.324959  0.445975  0.284070  0.104584
ENPH    0.381484  0.324959  1.000000  0.359787  0.165575  0.027988
FTNT    0.567486  0.445975  0.359787  1.000000  0.276838  0.165931
MCD     0.321937  0.284070  0.165575  0.276838  1.000000  0.147199
PFE     0.118987  0.104584  0.027988  0.165931  0.147199  1.000000


##MODIFIED CORRELATION

In [ ]:
corr_matrix_inverted = risk_models.cov_to_corr(cov_matrix_inverted)
print(corr_matrix_inverted)

Ticker      ADBE      AMZN      ENPH      FTNT       MCD       PFE
Ticker                                                            
ADBE    1.000000 -0.613254 -0.381484 -0.567486 -0.321937  0.118987
AMZN   -0.613254  1.000000  0.324959  0.445975  0.284070 -0.104584
ENPH   -0.381484  0.324959  1.000000  0.359787  0.165575 -0.027988
FTNT   -0.567486  0.445975  0.359787  1.000000  0.276838 -0.165931
MCD    -0.321937  0.284070  0.165575  0.276838  1.000000 -0.147199
PFE     0.118987 -0.104584 -0.027988 -0.165931 -0.147199  1.000000


# Mean-Variance Efficient Portfolio

##1)Optimization 1 (orginal er, cox matrix --> to find weights to rescale)

In [ ]:
ef = EfficientFrontier(expected_returns_series, cov_matrix, weight_bounds = (-1,1))
ef.max_sharpe()

weights = ef.clean_weights()
print(weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('ADBE', -1.0), ('AMZN', 0.60904), ('ENPH', 0.2454), ('FTNT', 0.50404), ('MCD', 1.0), ('PFE', -0.35848)])
Expected annual return: 53.5%
Annual volatility: 39.5%
Sharpe Ratio: 1.30


(0.5346062226729995, 0.3947037498153074, 1.3037783981373312)

In [ ]:
rescaled = ef.weights / np.abs(ef.weights).sum()
print(np.abs(ef.weights).sum())
np.abs(rescaled).sum()  # = 1.000000
print(rescaled)

3.716963331811177
[-0.26903682  0.16385363  0.06602287  0.13560509  0.26903682 -0.09644477]


In [ ]:
from pypfopt import base_optimizer

perfomance_portafoglio_finale = base_optimizer.portfolio_performance(
    rescaled, expected_returns_series, cov_matrix, verbose=True, risk_free_rate=0.02
)
print(perfomance_portafoglio_finale)

Expected annual return: 14.4%
Annual volatility: 10.6%
Sharpe Ratio: 1.17
(0.1438287588412932, 0.10618984223957326, 1.1661073811742282)


##2)OPTIMIZATION 2(er all positive, cov_matrix modified)

In [ ]:
ef_inverted = EfficientFrontier(expected_returns_series_inverted, cov_matrix_inverted, weight_bounds = (0,1))
ef_inverted.max_sharpe()

weights_allpositive = ef_inverted.clean_weights()
print(weights_allpositive)
ef_inverted.portfolio_performance(verbose=True)

OrderedDict([('ADBE', 0.24502), ('AMZN', 0.07417), ('ENPH', 0.04728), ('FTNT', 0.09022), ('MCD', 0.32939), ('PFE', 0.21392)])
Expected annual return: 15.2%
Annual volatility: 9.6%
Sharpe Ratio: 1.37


(0.1518127951365767, 0.09593494050353708, 1.373981100574267)

###change of sign of the asset that we want to short (in accordance to our expectation)

In [ ]:
weights_longshort = weights_allpositive.copy()

# Inverti il segno solo nelle colonne ADBE e PFE
weights_longshort['ADBE'] = -weights_longshort['ADBE']
weights_longshort['PFE'] = -weights_longshort['PFE']

# Visualizza i pesi con il segno invertito
print(weights_longshort)

OrderedDict([('ADBE', -0.24502), ('AMZN', 0.07417), ('ENPH', 0.04728), ('FTNT', 0.09022), ('MCD', 0.32939), ('PFE', -0.21392)])


test to see if by using the original er, cov_matrix and weights long/short we get the same return

In [ ]:
from pypfopt import base_optimizer

perfomance_portafoglio_finale = base_optimizer.portfolio_performance(
    weights_longshort, expected_returns_series, cov_matrix, verbose=True, risk_free_rate=0.02
)
print(perfomance_portafoglio_finale)

Expected annual return: 15.2%
Annual volatility: 9.6%
Sharpe Ratio: 1.37
(0.1518122, 0.09593450738733082, 1.373981100125055)
